In [3]:
import kagglehub
import os

path = kagglehub.dataset_download("manideep1108/tusimple")

print("Data Svaed:", path)
print(os.listdir(path))

Using Colab cache for faster access to the 'tusimple' dataset.
Data Svaed: /kaggle/input/tusimple
['TUSimple', 'test_label_new.json']


In [1]:
import json
import cv2
import numpy as np
import os
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print(os.listdir(path))

['TUSimple', 'test_label_new.json']


In [5]:
tusimple = os.path.join(path, "TUSimple")
print(os.listdir(tusimple))
train_set_dir = os.path.join(tusimple, "train_set")
print(os.listdir(train_set_dir))


['test_set', 'test_label.json', 'train_set']
['readme.md', 'clips', 'label_data_0601.json', 'label_data_0313.json', 'label_data_0531.json', 'seg_label']


In [25]:
def create_mask(json_file,root, size):
  raw_path = json_file["raw_file"]
  img_path = os.path.join(root, raw_path.lstrip('/'))

  if not os.path.exists(img_path):
        return None, None

  img = cv2.imread(img_path)
  if img is None:
     return None, None
  lanes = json_file['lanes']
  h_samples = json_file['h_samples']
  mask = np.zeros((img.shape[0], img.shape[1]),dtype=np.uint8)
  for lane in lanes:
    point = []
    for x, y in zip(lane, h_samples):
      if x > 0:
        point.append([x, y])
    if len(point) > 0:
      point = np.array([point], dtype=np.int32)
      cv2.polylines(mask, point, isClosed=False, color=255, thickness=10)
    img_resized = cv2.resize(img,size)
    mask_resized = cv2.resize(mask, size, interpolation=cv2.INTER_NEAREST)
  return img_resized,mask_resized

In [15]:
json_files = [
    'label_data_0601.json',
    'label_data_0313.json',
    'label_data_0531.json'
]
size = (512,256)


In [24]:
output_dir_drive = '/content/drive/MyDrive/Unet/ProcessedImages'
images_out_dir = os.path.join(output_dir_drive, 'images')
masks_out_dir = os.path.join(output_dir_drive, 'masks')
os.makedirs(images_out_dir, exist_ok=True)
os.makedirs(masks_out_dir, exist_ok=True)


In [ ]:
def process_data(json_file, root):
  full_json_path = os.path.join(root, json_file)
  with open(full_json_path, 'r') as f:
    lines = f.readlines()
  for line in tqdm(lines):
    try:
      json_data = json.loads(line)
      img, mask = create_mask(json_data, root, size)
      file_id = json_data["raw_file"].replace('/', '_').replace('.jpg', '')
      cv2.imwrite(os.path.join(images_out_dir, f"{file_id}.jpg"), img)
      cv2.imwrite(os.path.join(masks_out_dir, f"{file_id}.png"), mask)

    except Exception as e:
        print(f"\n Error file {json_data["raw_file"]}: {e}")
        continue

In [27]:
for j_file in json_files:
    process_data(j_file, train_set_dir)

100%|██████████| 358/358 [00:21<00:00, 16.95it/s]
